In [1]:
# coding: utf-8

# our all
import numpy as np
import pandas as pd

# usefull pandas settings
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

# for API working and current time
import requests
import datetime

# chess pgn-reading tool
from pgn_parser import pgn, parser

# multistreaming
import threading

# отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')

In [2]:
user_list=['andreyvict', 'sever043', 'Rosolimo']
user=user_list[2]

In [3]:
user

'Rosolimo'

In [5]:
df=pd.read_csv('data/cur_user.csv')

In [6]:
df.sample(4)

,Unnamed: 0,rules,time_class,time_control,rated,game,result,date,time,period,white.username,black.username,white.rating,black.rating,white.result,black.result,white.score,black.score,white_elo_forecast,black_elo_forecast,eco,eco_url,game_url,white_url,black_url,target_user,score,rating,opponent_rating,base_elo_forec,color,year,month
44960,0,chess,blitz,300,True,1,1-0,2014-07-27,21:06:47,2014-7,Yefimus1941,Rosolimo,1517,1519,win,resigned,1.0,0.0,0.497122,0.502878,A45,https://www.chess.com/openings/A45-Indian-Game,https://www.chess.com/live/game/871606814,https://api.chess.com/pub/player/yefimus1941,https://api.chess.com/pub/player/rosolimo,Rosolimo,0.0,1519,1517,0.502878,b,2014,7
47509,0,chess,blitz,180,True,1,0-1,2019-02-10,17:17:24,2019-2,Milansss3,AndrewSK76,1866,1901,resigned,win,0.0,1.0,0.449801,0.550199,A00,https://www.chess.com/openings/A00-Hungarian-Opening-1...d5,https://www.chess.com/live/game/3448362745,https://api.chess.com/pub/player/milansss3,https://api.chess.com/pub/player/andrewsk76,Rosolimo,1.0,1901,1866,0.550199,b,2019,2
53589,0,chess,blitz,300+5,True,1,0-1,2018-12-22,07:04:36,2018-12,na77ator,slang8,1467,1613,timeout,win,0.0,1.0,0.301441,0.698559,B13,https://www.chess.com/openings/B13-Caro-Kann-Defense-Exchange-Variation-3...cxd5-4.Nf3-Nf6,https://www.chess.com/live/game/3313499588,https://api.chess.com/pub/player/na77ator,https://api.chess.com/pub/player/slang8,Rosolimo,1.0,1613,1467,0.698559,b,2018,12
8512,0,chess,blitz,600,True,1,0-1,2015-09-14,14:16:26,2015-9,chubarin,madrid7676,936,980,resigned,win,0.0,1.0,0.437015,0.562985,B00,https://www.chess.com/openings/B00-Nimzowitsch-Defense-2.d4-e6,https://www.chess.com/live/game/1280465584,https://api.chess.com/pub/player/chubarin,https://api.chess.com/pub/player/madrid7676,Rosolimo,1.0,980,936,0.562985,b,2015,9
